In [1]:
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K
import os
import cv2
import glob
import PIL
import shutil
import numpy as np
import pandas as pd
from Datagenerators import *
from Arch_AttentionUnet import *
from metrics import *

# ml libs
from tensorflow import keras
import keras.backend as K
from keras.callbacks import CSVLogger
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers.experimental import preprocessing

np.set_printoptions(precision=3, suppress=True)

CSVLogger.on_test_begin = CSVLogger.on_train_begin
CSVLogger.on_test_batch_end = CSVLogger.on_epoch_end
CSVLogger.on_test_end = CSVLogger.on_train_end

In [2]:
model = build_attention_unet(n_channels=3, ker_init='he_normal', dropout=0.2)
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4), dice_coef, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing] )

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 16  2320        ['conv2d[0][0]']                 
                                )                                                             

 conv2d_17 (Conv2D)             (None, 16, 16, 64)   8256        ['conv2d_16[0][0]']              
                                                                                                  
 activation_3 (Activation)      (None, 16, 16, 64)   0           ['conv2d_17[0][0]']              
                                                                                                  
 conv2d_19 (Conv2D)             (None, 16, 16, 64)   4160        ['activation_3[0][0]']           
                                                                                                  
 conv2d_transpose_1 (Conv2DTran  (None, 16, 16, 64)  36928       ['conv2d_19[0][0]']              
 spose)                                                                                           
                                                                                                  
 conv2d_18 (Conv2D)             (None, 16, 16, 64)   16448       ['conv2d_5[0][0]']               
          

                                                                                                  
 conv2d_30 (Conv2D)             (None, 64, 64, 32)   9248        ['dropout_4[0][0]']              
                                                                                                  
 conv2d_31 (Conv2D)             (None, 64, 64, 16)   528         ['conv2d_30[0][0]']              
                                                                                                  
 activation_9 (Activation)      (None, 64, 64, 16)   0           ['conv2d_31[0][0]']              
                                                                                                  
 conv2d_33 (Conv2D)             (None, 64, 64, 16)   272         ['activation_9[0][0]']           
                                                                                                  
 conv2d_transpose_3 (Conv2DTran  (None, 64, 64, 16)  2320        ['conv2d_33[0][0]']              
 spose)   

In [3]:
directory = [f.path for f in os.scandir(PATH) if f.is_dir()]


def generate_ids(dirLst):
    ids = []
    for i in range(0,len(dirLst)):
        ids.append(dirLst[i][dirLst[i].rfind('/')+1:])
    return ids

train_and_test_ids = generate_ids(directory); 

    
train_test_ids, val_ids = train_test_split(train_and_test_ids, train_size=236, test_size=60, random_state=14) 
train_ids, test_ids = train_test_split(train_test_ids, test_size=36, random_state=14) 

In [4]:
training_generator = DataGenerator2D(train_ids)
valid_generator = DataGenerator2D(val_ids)
test_generator = DataGenerator2D(test_ids)

In [5]:
csv_logger = CSVLogger('Logs/training/attU_net_2d.log', separator=',', append=False)

eval_csv_logger = CSVLogger('Logs/testing/attU_net_2d.log', separator=',', append=False)


callbacks = [
      keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1),
        csv_logger
    ]

In [6]:
K.clear_session()

history =  model.fit(training_generator,
                    epochs=100,
                    steps_per_epoch=len(train_ids),
                    callbacks= callbacks,
                    validation_data = valid_generator
                    )  
model.save("Models/attU_net_2d.h5")

Epoch 1/100
200/200 [==============================] - 83s 362ms/step - loss: 0.0779 - accuracy: 0.9827 - mean_io_u: 0.7842 - dice_coef: 0.2986 - dice_coef_necrotic: 0.1027 - dice_coef_edema: 0.2384 - dice_coef_enhancing: 0.0858 - val_loss: 0.1906 - val_accuracy: 0.9827 - val_mean_io_u: 0.4270 - val_dice_coef: 0.2668 - val_dice_coef_necrotic: 0.1065 - val_dice_coef_edema: 0.2337 - val_dice_coef_enhancing: 0.1055 - lr: 0.0010
Epoch 2/100
200/200 [==============================] - 54s 271ms/step - loss: 0.0500 - accuracy: 0.9869 - mean_io_u: 0.7918 - dice_coef: 0.3523 - dice_coef_necrotic: 0.1769 - dice_coef_edema: 0.3791 - dice_coef_enhancing: 0.1945 - val_loss: 0.1074 - val_accuracy: 0.9572 - val_mean_io_u: 0.7446 - val_dice_coef: 0.3439 - val_dice_coef_necrotic: 0.2171 - val_dice_coef_edema: 0.3244 - val_dice_coef_enhancing: 0.2402 - lr: 0.0010
Epoch 3/100
200/200 [==============================] - 53s 264ms/step - loss: 0.0378 - accuracy: 0.9887 - mean_io_u: 0.8009 - dice_coef: 0.412

Epoch 17/100
200/200 [==============================] - 53s 265ms/step - loss: 0.0235 - accuracy: 0.9925 - mean_io_u: 0.8084 - dice_coef: 0.5235 - dice_coef_necrotic: 0.3500 - dice_coef_edema: 0.6579 - dice_coef_enhancing: 0.6173 - val_loss: 0.1436 - val_accuracy: 0.9609 - val_mean_io_u: 0.7977 - val_dice_coef: 0.4035 - val_dice_coef_necrotic: 0.2576 - val_dice_coef_edema: 0.3707 - val_dice_coef_enhancing: 0.3325 - lr: 1.0000e-06
Epoch 18/100
200/200 [==============================] - 53s 264ms/step - loss: 0.0235 - accuracy: 0.9925 - mean_io_u: 0.8085 - dice_coef: 0.5238 - dice_coef_necrotic: 0.3500 - dice_coef_edema: 0.6577 - dice_coef_enhancing: 0.6180 - val_loss: 0.1494 - val_accuracy: 0.9590 - val_mean_io_u: 0.7971 - val_dice_coef: 0.4016 - val_dice_coef_necrotic: 0.2578 - val_dice_coef_edema: 0.3679 - val_dice_coef_enhancing: 0.3294 - lr: 1.0000e-06
Epoch 19/100
200/200 [==============================] - 53s 264ms/step - loss: 0.0235 - accuracy: 0.9925 - mean_io_u: 0.8084 - dice_

Epoch 36/100
200/200 [==============================] - 64s 320ms/step - loss: 0.0234 - accuracy: 0.9925 - mean_io_u: 0.8081 - dice_coef: 0.5254 - dice_coef_necrotic: 0.3520 - dice_coef_edema: 0.6604 - dice_coef_enhancing: 0.6201 - val_loss: 0.1497 - val_accuracy: 0.9590 - val_mean_io_u: 0.7972 - val_dice_coef: 0.4018 - val_dice_coef_necrotic: 0.2586 - val_dice_coef_edema: 0.3658 - val_dice_coef_enhancing: 0.3302 - lr: 1.0000e-06
Epoch 37/100
200/200 [==============================] - 60s 300ms/step - loss: 0.0234 - accuracy: 0.9925 - mean_io_u: 0.8081 - dice_coef: 0.5256 - dice_coef_necrotic: 0.3523 - dice_coef_edema: 0.6602 - dice_coef_enhancing: 0.6203 - val_loss: 0.1435 - val_accuracy: 0.9613 - val_mean_io_u: 0.7968 - val_dice_coef: 0.4045 - val_dice_coef_necrotic: 0.2566 - val_dice_coef_edema: 0.3702 - val_dice_coef_enhancing: 0.3346 - lr: 1.0000e-06
Epoch 38/100
200/200 [==============================] - 53s 265ms/step - loss: 0.0234 - accuracy: 0.9925 - mean_io_u: 0.8082 - dice_

Epoch 55/100
200/200 [==============================] - 55s 273ms/step - loss: 0.0232 - accuracy: 0.9926 - mean_io_u: 0.8086 - dice_coef: 0.5274 - dice_coef_necrotic: 0.3541 - dice_coef_edema: 0.6627 - dice_coef_enhancing: 0.6232 - val_loss: 0.1389 - val_accuracy: 0.9628 - val_mean_io_u: 0.7979 - val_dice_coef: 0.4065 - val_dice_coef_necrotic: 0.2564 - val_dice_coef_edema: 0.3713 - val_dice_coef_enhancing: 0.3381 - lr: 1.0000e-06
Epoch 56/100
200/200 [==============================] - 53s 264ms/step - loss: 0.0232 - accuracy: 0.9926 - mean_io_u: 0.8087 - dice_coef: 0.5273 - dice_coef_necrotic: 0.3549 - dice_coef_edema: 0.6630 - dice_coef_enhancing: 0.6232 - val_loss: 0.1347 - val_accuracy: 0.9643 - val_mean_io_u: 0.7976 - val_dice_coef: 0.4084 - val_dice_coef_necrotic: 0.2541 - val_dice_coef_edema: 0.3737 - val_dice_coef_enhancing: 0.3416 - lr: 1.0000e-06
Epoch 57/100
200/200 [==============================] - 53s 264ms/step - loss: 0.0232 - accuracy: 0.9926 - mean_io_u: 0.8087 - dice_

Epoch 74/100
200/200 [==============================] - 52s 261ms/step - loss: 0.0231 - accuracy: 0.9926 - mean_io_u: 0.8098 - dice_coef: 0.5293 - dice_coef_necrotic: 0.3564 - dice_coef_edema: 0.6651 - dice_coef_enhancing: 0.6250 - val_loss: 0.1462 - val_accuracy: 0.9606 - val_mean_io_u: 0.7988 - val_dice_coef: 0.4049 - val_dice_coef_necrotic: 0.2576 - val_dice_coef_edema: 0.3670 - val_dice_coef_enhancing: 0.3345 - lr: 1.0000e-06
Epoch 75/100
200/200 [==============================] - 52s 260ms/step - loss: 0.0230 - accuracy: 0.9926 - mean_io_u: 0.8097 - dice_coef: 0.5291 - dice_coef_necrotic: 0.3569 - dice_coef_edema: 0.6652 - dice_coef_enhancing: 0.6258 - val_loss: 0.1589 - val_accuracy: 0.9564 - val_mean_io_u: 0.7993 - val_dice_coef: 0.4011 - val_dice_coef_necrotic: 0.2609 - val_dice_coef_edema: 0.3641 - val_dice_coef_enhancing: 0.3249 - lr: 1.0000e-06
Epoch 76/100
200/200 [==============================] - 52s 259ms/step - loss: 0.0230 - accuracy: 0.9926 - mean_io_u: 0.8099 - dice_

Epoch 93/100
200/200 [==============================] - 52s 260ms/step - loss: 0.0229 - accuracy: 0.9927 - mean_io_u: 0.8107 - dice_coef: 0.5309 - dice_coef_necrotic: 0.3593 - dice_coef_edema: 0.6679 - dice_coef_enhancing: 0.6277 - val_loss: 0.1519 - val_accuracy: 0.9588 - val_mean_io_u: 0.8009 - val_dice_coef: 0.4037 - val_dice_coef_necrotic: 0.2593 - val_dice_coef_edema: 0.3639 - val_dice_coef_enhancing: 0.3307 - lr: 1.0000e-06
Epoch 94/100
200/200 [==============================] - 52s 259ms/step - loss: 0.0229 - accuracy: 0.9927 - mean_io_u: 0.8106 - dice_coef: 0.5310 - dice_coef_necrotic: 0.3597 - dice_coef_edema: 0.6678 - dice_coef_enhancing: 0.6287 - val_loss: 0.1522 - val_accuracy: 0.9586 - val_mean_io_u: 0.8007 - val_dice_coef: 0.4032 - val_dice_coef_necrotic: 0.2605 - val_dice_coef_edema: 0.3638 - val_dice_coef_enhancing: 0.3300 - lr: 1.0000e-06
Epoch 95/100
200/200 [==============================] - 52s 260ms/step - loss: 0.0229 - accuracy: 0.9927 - mean_io_u: 0.8108 - dice_

In [ ]:
# model = keras.models.load_model('Models/2D/attU_net_2d.h5', 
#                                    custom_objects={ 'accuracy' : tf.keras.metrics.MeanIoU(num_classes=4),
#                                                    "dice_coef": dice_coef,
#                                                    "dice_coef_necrotic": dice_coef_necrotic,
#                                                    "dice_coef_edema": dice_coef_edema,
#                                                    "dice_coef_enhancing": dice_coef_enhancing
#                                                   }, compile=True)

In [ ]:
model.evaluate(test_generator, callbacks=eval_csv_logger, verbose=1)

25/36 [===================>..........] - ETA: 2s - loss: 0.0904 - accuracy: 0.9726 - mean_io_u: 0.7999 - dice_coef: 0.4715 - dice_coef_necrotic: 0.3112 - dice_coef_edema: 0.4753 - dice_coef_enhancing: 0.5422

In [ ]:
valid_csv_logger = CSVLogger('Logs/validation/attU_net_2d.log', separator=',', append=False)
model.evaluate(valid_generator, callbacks=valid_csv_logger, verbose=1)